# Sección 1: Carga y manipulación de datos en GPU con RAPIDS

## Carga de datos

In [ ]:
import os
import os.path

import cudf
import cupy as cp

%run ../utils/f_static_data.py
%run ../utils/f_northing.py
%run ../utils/f_northing_numpy.py
%run ../utils/f_utils.py

Según el tamaño de muestra que deseemos estudiar, podemos cargar uno o múltiples datasets en memoria. Para evitar discrepancias en el número de columnas, seleccionamos manualmente las que nos interesan.

Los datos han sido obtenidos del repositorio público de AirBnB: http://insideairbnb.com/get-the-data.html, por cada ciudad se utilizan todos los datasets de listados (listings.tar.gz) disponibles para el año 2020.

In [ ]:
cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
#cities_to_use = cities_to_use_1()
#cities_to_use = cities_to_use_2()

columns_to_use = ['host_id', 'host_response_rate', 'host_acceptance_rate', 'latitude', 'longitude', 
                  'accommodates', 'price', 'number_of_reviews', 'reviews_per_month', 'neighbourhood_cleansed']

Debido a la cantidad de diferentes ficheros que debemos leer, y a que los diferentes _scraping_ realizados por el equipo(s) de AirBnB no siempre son iguales, podemos encontrar que en el dataset de algunos meses para algunas ciudades faltan columnas.

La siguiente celda recorre las ciudades disponibles y comprueba en qué datasets faltan columnas, dado que las trazas de error de RAPIDS y pandas no siempre son detalladas. El resultado esperado es que no se imprima ninguna línea por la consola, ya que significaría que todos los datasets disponibles tienen las columnas necesarias.

Cargamos en memoria los datasets que vayamos a utilizar y los compilamos en uno. Convertimos columnas con tipos de datos dispares de modo que puedan ser almacenadas en un solo DataFrame.

In [ ]:
%%time
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings.size == 0:
                    listings = temp_df
                else:
                    for column in listings.columns:
                        if listings[column].dtype != temp_df[column].dtype:
                            print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                    listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates().reset_index(drop=True)
listings.shape

La instrucción nvidia-smi nos permite leer la información de uso de la(s) GPU(s) disponible(s). En particular, es útil para controlar cuánta memoria de vídeo (VRAM) está en uso. Dado que RAPIDS carga todos los datos en la GPU para optimizar el acceso a la información, hay que controlar que haya suficiente memoria antes de leer datos.

In [ ]:
!nvidia-smi

## Tratamiento de datos con cuDF y cuPY

Convertimos todas las columnas a un tipo de datos común: float32 es el más aceptado por los algoritmos de RAPIDS.

Dado que algunos algoritmos no suportan datos no numéricos, factorizamos las columnas de texto. La operación factorize() convierte una columna con valores no numéricos a un mapa en el que cada valor único se representa por un integer.

In [ ]:
%time
type_conversion(listings, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings, ['neighbourhood_cleansed'])

En columnas con texto no deseado (como precios con símbolos de moneda), limpiamos caracteres no deseados y convertimos valores nulos. Finalmente, convertimos todo a dtype float32 para mantener la consistencia.

In [ ]:
%%time
clean_format_strings(listings, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price(listings, ['price'])

## Generación de datos geográficos

Convertimos las coordenadas de longitud y latitud a distancias norte y este, a fin de normalizar la escala de los gráficos. La función de conversión (con su fuente de referencia) se encuentra en un fichero de utilidad aparte.

In [ ]:
%%time
cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

Resultado final del tratamiento de datos:

In [ ]:
listings.dtypes

In [ ]:
listings.head()

# Sección 2: Carga y manipulación de datos en CPU con pandas y numPy

## Carga de datos

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%time
listings_cpu = pd.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df_cpu = pd.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df_cpu, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings_cpu.size == 0:
                    listings_cpu = temp_df_cpu
                else:
                    for column in listings_cpu.columns:
                        if listings_cpu[column].dtype != temp_df_cpu[column].dtype:
                            print('Found error: '+column+' type '+listings_cpu[column].dtype.name+' doesnt match '+temp_df_cpu[column].dtype.name)
                    listings_cpu = listings_cpu.append(temp_df_cpu)
                    
listings_cpu = listings_cpu.drop_duplicates().reset_index(drop=True)
listings_cpu.shape

## Tratamiento de datos

In [ ]:
%time
type_conversion(listings_cpu, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings_cpu, ['neighbourhood_cleansed'])

In [ ]:
%%time
clean_format_strings(listings_cpu, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price_cpu(listings_cpu, ['price'])

## Generación de datos geográficos

In [ ]:
%%time
numpy_lat = listings_cpu['latitude'].to_numpy()
numpy_long = listings_cpu['longitude'].to_numpy()
n_numpy_array, e_numpy_array = latlong2osgbgrid_numpy(numpy_lat, numpy_long)
listings_cpu['northing'] = pd.Series(n_numpy_array).astype('float32')
listings_cpu['easting'] = pd.Series(e_numpy_array).astype('float32')

In [ ]:
listings.dtypes

In [ ]:
listings.head()

El comando %reset nos permite limpiar todas las variables sin reiniciar el kernel. Esto nos permite aprovechar al máximo la gestión de memoria inicializada por RAPIDS, ejecutando pruebas múltiples veces en el mismo kernel inicializado.

In [ ]:
%reset -f

# Sección 3: Visualización de resultados mediante cuXfilter

La librería cuXfilter nos permite visualizar datos gráficamente. Vamos a visualizar los listados de AirBnB en Sevilla a 29 de octubre de 2020, con un selector por zonas.

Para ver el gráfico sin abrir un widget podemos usar el siguiente comando:

Para detener la ejecución del gráfico, podemos usar el siguiente comando: